In [ ]:
#import dependencies
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np

# chisom
import hvplot.pandas
import requests
import warnings

from sklearn import datasets
# from datetime import date
import datetime as dt

# store filepaths into variable
results_csv = pd.read_csv('Resources/results.csv', na_values=r'\N')
lap_times_csv = pd.read_csv('Resources/lap_times.csv', na_values=r'\N')
races_csv = pd.read_csv('Resources/races.csv', na_values=r'\N')
constructors_csv = pd.read_csv('Resources/constructors.csv', na_values=r'\N')
constructor_results = pd.read_csv('Resources/constructor_results.csv', na_values=r'\N')
constructor_standings = pd.read_csv('Resources/constructor_standings.csv', na_values=r'\N')
drivers_csv = pd.read_csv('Resources/drivers.csv', na_values=r'\N')
drivers_standings_csv = pd.read_csv('Resources/driver_standings.csv', na_values=r'\N')
circuits_csv = pd.read_csv('Resources/circuits.csv', na_values=r'\N')
pit_stop = pd.read_csv('Resources/pit_stops.csv', na_values=r'\N')

In [ ]:
#merge constructor results and constructor standings csv
constructor_df = pd.merge(constructor_results, constructor_standings, how="inner")
constructor_df


In [ ]:
#merge the combined construtors with the constructors csv
constructor_df = pd.merge(constructor_df, constructors_csv, how="inner")
constructor_df

In [ ]:
#drop extra columns and combine the merged constructors with races csv on "raceId"

new_constructor_df_constructor= constructor_df.drop(columns=["status", "positionText", "constructorRef"])
new_constructor_df = pd.merge(constructor_df, races_csv, on="raceId")
new_constructor_df

In [ ]:
#dropping extra columns again after last merge
clean_construtors= new_constructor_df.drop(columns=["fp1_date", "fp1_time", "fp2_date",	"fp2_time", "fp3_date", "fp3_time","quali_date","quali_time","sprint_date", "sprint_time","date","time", "url_x", "url_y", "status","positionText"])
clean_construtors

In [ ]:
#rename columns
clean_construtors = clean_construtors.rename(columns={"name_x":"constructor_name", "name_y":"circuit name"})
clean_construtors

In [ ]:
#find winning constructors from 2004

constructors_after_2004 = clean_construtors[(clean_construtors["year"]) >=2004]
constructors_after_2004 = constructors_after_2004.sort_values("year", ascending=True)
constructors_after_2004

In [ ]:
top_winning_constructors = constructors_after_2004[constructors_after_2004["position"]==1].groupby(["constructorId", "constructor_name","nationality"])
top_winning_constructors.head()

In [ ]:
#calculate win counts for each year 
win_counts = top_winning_constructors['wins'].sum().reset_index()
win_counts.value_counts()


In [ ]:
# Create a bar chart for top constructors per year from 2004
plt.figure(figsize=(10, 6))
plt.bar(win_counts['constructor_name'], win_counts['wins'])
plt.xlabel('Constructors')
plt.ylabel('Total Wins')
plt.title('Most Constructor Wins since 2004')
plt.xticks(rotation=45)
plt.tight_layout()

# Show the chart
plt.show()

In [ ]:
# Create a pie chart for top constructors per year based on nationality
fig, ax = plt.subplots(figsize=(8, 8))
ax.pie(win_counts['wins'], labels=win_counts['nationality'], autopct='%1.1f%%', startangle=90)

ax.set_title('Winning Constructor Nationality Per Year from 2004')

plt.axis('equal')  

plt.tight_layout()
plt.show()

In [ ]:
#based on the data, the top 3 most successful constructors are Ferrari, Redbull and Mercedes. Ferarri has been the most successful constructor for f1 in the last 20 years. There can be many factors: such as investment of resources, technical innovation and talent recruitment. Italian made cars serve as a blueprint for success for team managers starting a team in F1. 

In [ ]:
races_df=pd.DataFrame(races_csv, columns=[
                        "year", "raceId", "date"])
races_df

In [ ]:
results_df=pd.DataFrame(results_csv, columns=[
                        "raceId", "driverId"])
results_df

In [ ]:
drivers_df=pd.DataFrame(drivers_csv, columns=[
                        "dob", "driverRef", "driverId"])
drivers_df

In [ ]:
drivers_standings_df=pd.DataFrame(drivers_standings_csv, columns=[
                        "wins", "driverId", "raceId"])
drivers_standings_df

In [ ]:
driver_merged=pd.merge(drivers_df, drivers_standings_df,  on="driverId")

In [ ]:
driver_merged=pd.merge(drivers_df, drivers_standings_df,  on="driverId")
driver_merged_clean= driver_merged.dropna()
driver_merged_clean.head()

In [ ]:
driver_merged_clean=pd.merge(driver_merged_clean, races_df, on="raceId")
driver_merged_clean


In [ ]:
driver_name = driver_merged_clean.set_index('driverRef').groupby(['driverRef'])
driver_name.head()
after_2004_races_df = driver_merged_clean[(driver_merged_clean['date']) > '2004-01-01']
after_2004_races_df= after_2004_races_df.sort_values('year', ascending=True)
after_2004_races_df

In [ ]:
after_2004_races_df.count()

In [ ]:
after_2004_races_df["dob"] = pd.to_datetime(after_2004_races_df["dob"])

# Extract the year from the date of birth
after_2004_races_df["dob_year"] = after_2004_races_df["dob"].dt.year

# Calculate the age at the time of the race in years
after_2004_races_df["age"] = after_2004_races_df["year"] - after_2004_races_df["dob_year"]

# Drop the temporary 'dob_year' column if you no longer need it
after_2004_races_df = after_2004_races_df.drop(columns=["dob_year"])
after_2004_races_df

In [ ]:
reduce_driver_merged_group = after_2004_races_df[(after_2004_races_df["age"] < 60)]
reduce_driver_merged_group .head()

#explain why 60 was the cut off mark


The cut off mark for the age was based on the retirement age in the united states, most career focused job have s retirment age bewteen 50-65 years and i believe having a cut off age at 60 was a good benchamrk 

In [ ]:
reduce_driver_merged_group.count()

In [ ]:
driver_merged_group2=reduce_driver_merged_group.groupby(["wins"]).max()
driver_merged_group2.tail()

In [ ]:
driver_merged_group2_aver=reduce_driver_merged_group['age'].mean()
driver_merged_group2_aver
#this result shows that the average age bewteen all drivers regardless of the amount of wins is 28 years old 

In [ ]:
driver_merged_group2_avera=reduce_driver_merged_group['wins'].mean()
driver_merged_group2_avera

In [ ]:
age_group= reduce_driver_merged_group.groupby(["age"])
age_group.head()


In [ ]:
top4_wins=reduce_driver_merged_group.loc[reduce_driver_merged_group['wins']>=10]
top4_wins
#explain why you use 12 as a benchmark
#Looking at the top 4 winners, i needed a benchmark to see what age the drivers won. Looking at 10 wins seem the more reasonable because the most wins should be drivers that have double digts wins, with the amount of race held yearly and the amount ciructs done during the year.

Looking at the top 4 winners, i needed a benchmark to see what age the drivers won. Looking at 10 wins seem the more reasonable because the most wins should be drivers that have double digts wins, with the amount of race held yearly and the amount ciructs done during the year.

In [ ]:
top4_wins_group= top4_wins.groupby(["driverRef"])
top4_wins_group

In [ ]:
top4_wins_aver=top4_wins_group['age'].mean()
top4_wins_aver

In [ ]:
age_groups=reduce_driver_merged_group.groupby(["age"]).max()

In [ ]:
age_of_driver_graph= age_groups['wins']
age_of_driver_graph.plot(kind="bar", figsize=(20,3.5))
plt.xlabel("age")
plt.ylabel("Wins")
plt.title("Wins & Age")
plt.show()

In [ ]:
winss= reduce_driver_merged_group['wins']
age = reduce_driver_merged_group['age']
st.pearsonr(reduce_driver_merged_group['age'],reduce_driver_merged_group['wins'])
print(f"The correlation coefficient between wins and age is {round(st.pearsonr(winss, age)[0],2)}")
#explain how there is no correlation between the age and wins 


The correlation shown proves that there is a signficant but not enough signficance for the age of a driver to be taken into account when looking at the fsctors that play into whether or not a head coach should hire an young driver or someone of an older age.

In [ ]:
x_values = reduce_driver_merged_group['age']
y_values = reduce_driver_merged_group['wins']
linregress(x_values, y_values)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

In [ ]:
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(0,50),fontsize=15,color="red")
plt.xlabel('Age')
plt.ylabel('Wins')
plt.title("Wins & Age")
print(f"The r-squared is: {rvalue**2}")
plt.show()

# Pit Stop Analysis

In [ ]:

#merge pit_stops and results
df = pd.merge(pit_stop, results_csv, on=['raceId', 'driverId'], suffixes=('_pit', '_result'), how='inner')
# Convert milliseconds to numeric
df['milliseconds_pit'] = pd.to_numeric(df['milliseconds_pit'], errors='coerce')
df['milliseconds_result'] = pd.to_numeric(df['milliseconds_result'], errors='coerce')
# Convert milliseconds to seconds
df['seconds_pit'] = df['milliseconds_pit'] / 1000
df['seconds_result'] = df['milliseconds_result'] / 1000
# drop the columns that are not needed for the analysis
df.drop(['milliseconds_pit', 'milliseconds_result',  'fastestLap', 'fastestLapSpeed', 'statusId', 'laps', 'lap', 'resultId', 'fastestLapTime', 'fastestLapSpeed', 'statusId','number', 'grid',], axis=1, inplace=True)

print(df.columns)


## Correlation Analysis Between Pit Stop Duration and Final Race Result:
To understand the relationship between the duration of a pit stop and the final race result, a correlation analysis was performed on the cleaned dataset.

### Findings:
- The correlation coefficient was found to be 0.33462239711643876. This positive value indicates that as the duration of a pit stop increases, there's a tendency for the race result duration to also increase. However, it's essential to note that this is a moderate correlation, which means that while there is a relationship, there are also other factors influencing the final race result.
### Visualization of the Relationship:
A scatter plot was generated to visually represent the relationship between pit stop duration and race result duration:
- The scatter plot reinforces our understanding of the correlation. While there's a visible trend showing that longer pit stops are associated with longer race durations, the data points are somewhat dispersed, highlighting the influence of other factors in a race.
### Implications:
- Given the moderate positive correlation, it suggests that efficient pit stops can contribute to better race results. Although pit stop duration is just one of many factors that determine race outcomes, its influence is undeniable. In the context of winning an F1 championship, teams that consistently achieve shorter pit stop durations may have an advantage in races, potentially leading to better race positions and higher championship points.



In [ ]:
# Drop rows with missing values in either pit stop or result duration
df_cleaned = df.dropna(subset=['seconds_pit', 'seconds_result'])

# Calculate correlation
correlation = df_cleaned['seconds_pit'].corr(df_cleaned['seconds_result'])

print(f"Correlation between pit stop duration and final race result: {correlation}")

In [ ]:
# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(df_cleaned['seconds_pit'], df_cleaned['seconds_result'])
plt.title('Pit Stop Duration vs. Race Result Duration')
plt.xlabel('Pit Stop Duration (seconds)')
plt.ylabel('Race Result Duration (seconds)')
plt.show()


## ANOVA Test:
The ANOVA test was conducted to determine if there are significant differences in pit stop durations among various F1 constructors (teams). The result of the test provides two key metrics:

- F-statistic (7.61): This number represents the ratio of variance between the groups (different constructors) to the variance within the groups (intra-constructor variance). A higher F-statistic suggests that the differences between the group means are more substantial than what might be expected by chance.
- p-value (3.74e-23): This incredibly small p-value, way below the 0.05 threshold, tells us the probability of observing these results if the null hypothesis were true. Given its minuscule magnitude, it means the results are highly statistically significant.

From the above ANOVA result, we can infer:

- Significant Differences: There are significant differences in pit stop durations across different constructors. Not every team is performing pit stops with the same efficiency.
- Statistical Significance: The differences are not merely by chance; they are statistically significant, implying real variations in pit stop efficiencies among the teams.


The ANOVA results have crucial implications:

- Strategic Importance: The differences in pit stop durations among constructors highlight a potential area for competitive advantage. Teams with quicker pit stops can gain crucial seconds during a race, which can be the difference between securing a podium position or not.
- Season-long Impacts: Over a season, these seconds saved during pit stops can cumulate. A team with a consistent edge in pit stop efficiency might end up with more championship points due to better race finishes.
- Prioritization by Teams: If a team is lagging in pit stop efficiency, it might be a critical area to focus on. Enhancing their pit stop times could significantly boost their chances in individual races and the championship overall.

### Implications:
The ANOVA results underscore the significance of pit stop durations in F1 racing. Teams with a strategic advantage in pit stops stand a better chance of clinching better race positions and, consequently, the championship. As the season progresses, this advantage can be a defining factor in the title race. Any team serious about the championship should invest resources in optimizing their pit stop strategy and execution.

In [ ]:

# Extract pit stop durations by constructor for ANOVA
pit_durations = [df[df['constructorId'] == constructor]['seconds_pit'].dropna().values for constructor in df['constructorId'].unique()]

# Perform ANOVA
anova_result = st.f_oneway(*pit_durations)

# Display the ANOVA result
print("ANOVA Result:")
print(anova_result)

# Check for statistical significance
alpha = 0.05
if anova_result.pvalue < alpha:
    print("\nReject the null hypothesis. There are significant differences in pit stop durations among constructors.")
else:
    print("\nFail to reject the null hypothesis. There are no significant differences in pit stop durations among constructors.")

## Pit Stop Duration by Constructors:
To assess the efficiency of different F1 teams in their pit stops, a boxplot visualization was created to compare pit stop durations grouped by each constructor (or team).

### Visualization:

The boxplot shows the distribution of pit stop durations for each constructor. The colored boxes represent the interquartile range (25th percentile to 75th percentile) of pit stop durations, with the median (50th percentile) highlighted in black. Outliers (extremely short or long pit stops) were excluded from the visualization for clarity.
### Statistical Analysis:

To supplement the visualization and provide a deeper understanding, statistical descriptors for each constructor's pit stop durations were computed:

count: Number of pit stops recorded for the constructor.
mean: Average pit stop duration.
std: Standard deviation, showing the dispersion of the pit stop durations.
min: Shortest pit stop duration recorded.
25%: 25th percentile (lower quartile).
50%: Median (50th percentile).
75%: 75th percentile (upper quartile).
max: Longest pit stop duration recorded.
### Findings:

From the provided output, we can make several observations:

Constructors vary in the number of pit stops they've recorded, indicating different levels of participation or data availability.
The average pit stop duration (mean) varies across constructors. For instance, constructor 1 has an average pit stop duration of approximately 81.19 seconds, whereas constructor 166 averages around 24.24 seconds.
There's a significant range in pit stop durations, as indicated by the min and max values. Some teams have had exceedingly long pit stops, possibly due to technical issues.
The variability in pit stop durations (as denoted by the std values) also varies across teams. Some teams exhibit higher consistency in their pit stops than others.
### Implications:

The variation in pit stop durations among constructors implies that certain teams may possess an advantage in this aspect of the race. Teams with consistently shorter pit stops can potentially improve their race position, gaining crucial seconds that could translate into better race outcomes. In the context of the broader question regarding the influence of pit stops on winning an F1 championship, these insights indicate that the efficiency of pit stops can be a determinant factor. However, the impact of pit stop durations would still need to be analyzed in conjunction with other performance metrics to draw more comprehensive conclusions.

In [ ]:

fig, ax = plt.subplots(figsize=(15, 8))
df.boxplot(column='seconds_pit', by='constructorId', grid=False, showfliers=False, patch_artist=True, medianprops={'color':'black'}, ax=ax)
plt.title('Pit Stop Duration by Constructor')
plt.xlabel('Constructor ID')
plt.ylabel('Pit Stop Duration (seconds)')
plt.xticks(rotation=45, ha='right')
plt.show()

# Statistical Analysis
constructor_pit_duration_stats = df.groupby('constructorId')['seconds_pit'].describe()
print(constructor_pit_duration_stats) 

## Conclusion for Pit Stop Analysis:

In the high-stakes environment of Formula 1 racing, every fraction of a second matters. The in-depth analysis of pit stop durations across different constructors offers illuminating insights into the varying efficiency levels among teams.

Our findings reveal that there's a notable variation in pit stop durations across different constructors. This disparity suggests that while some teams have honed their pit stop process to a fine art, optimizing every movement for speed, others might have areas of improvement or might have faced unexpected issues leading to prolonged stops.

Given the close correlation between pit stop duration and race results, these variations cannot be overlooked. A team that consistently delivers fast and efficient pit stops can potentially leverage this advantage to improve their race position, which in turn can influence the championship outcomes.

However, while pit stop efficiency is undeniably a crucial component, it is just one of the myriad factors that determine a team's success in an F1 championship. Driver skill, car performance, race strategy, and external factors like weather conditions also play pivotal roles. But as this analysis underscores, in the realm of F1 racing, where races can be won or lost in milliseconds, mastering the art of the pit stop can be a game-changer.

In [ ]:
# Top Lap Speeds after 2004

In [ ]:
# working on results_csv, performing dropna
results_csv = results_csv.replace(to_replace="\\N", value=np.nan)
results_csv = results_csv.dropna()
results_csv

In [ ]:
# checking rows for raceId260 if there is empty data
results_csv.loc[results_csv['raceId']==260]

In [ ]:
# columns we are interested in
new_results_df = results_csv[['raceId', 'driverId', 'constructorId',
                         'position', 'laps', 'time', 'milliseconds', 
                         'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed']]
new_results_df['fastestLapTime'] = new_results_df['fastestLapTime'].astype("datetime64")
new_results_df = new_results_df.astype({"fastestLapTime": "datetime64"})
new_results_df['fastestLapTime'] = pd.to_datetime(new_results_df['fastestLapTime'],format).apply(lambda x: x.time())
new_results_df = new_results_df.replace(to_replace="\\N", value=np.nan)
new_results_df = new_results_df.dropna()
new_results_df.head(10)

In [ ]:
#checking types
new_results_df.dtypes

In [ ]:
#lap_times_csv
lap_times_csv = lap_times_csv.dropna()
lap_times_csv.head()

In [ ]:
#replacing null value to drop later
races_csv = races_csv.replace(to_replace="\\N", value=np.nan)
races_csv.head()

In [ ]:
# columns we are interested in
new_races_df = races_csv[['raceId', 'year', 'round', 'circuitId', 'name']]
new_races_df.head()

In [ ]:
# getting races after year 2004
after_2004_races_df = new_races_df[(new_races_df['year']) >= 2004]
after_2004_races_df= after_2004_races_df.sort_values('year', ascending=True)
after_2004_races_df.head()

In [ ]:
#Merging new_results_df, after_2004_races_df on raceId
merged_df = pd.merge(new_results_df, after_2004_races_df, on=['raceId'], how='inner')
merged_df= merged_df.sort_values("year", ascending=True)
merged_df["position"] = merged_df["position"].astype(float)
merged_df.head()

In [ ]:
# columns we are interested in to obtain driver names
drivers_csv = drivers_csv[['driverId', 'forename', 'surname']]
drivers_csv.head()

In [ ]:
# merging forename and surname to obtain driver name
drivers_csv['driver name'] = drivers_csv['forename'] + ' ' + drivers_csv['surname']
new_drivers_df = drivers_csv[['driverId', 'driver name']]
new_drivers_df

In [ ]:
#Merging merged_df, drivers_df on driverId
mega_merged_df = pd.merge(new_drivers_df, merged_df, on=['driverId'], how='outer')
mega_merged_df = mega_merged_df.sort_values('year', ascending=True)
mega_merged_df = mega_merged_df.dropna()
mega_merged_df = mega_merged_df.drop(columns=['round', 'rank'])
mega_merged_df.head()

In [ ]:
#checking data types
mega_merged_df.dtypes

In [ ]:
# columns we are interested in circuits_csv
new_circuits_df = circuits_csv[['circuitId', 'name', 'location', 'country', 'lat', 'lng']]
new_circuits_df.rename(columns = {'name':'Circuit Name'}, inplace = True)
new_circuits_df.head()

In [ ]:
#Merging mega_merged_df, new_circuits_df on circuitId
mega2_merged_df = pd.merge(mega_merged_df, new_circuits_df, on=['circuitId'], how='outer')
mega2_merged_df = mega2_merged_df.sort_values('year', ascending=True)
mega2_merged_df = mega2_merged_df.dropna()
mega2_merged_df.head()

In [ ]:
# getting columns we are interested from new_constructors_csv
new_constructors_df = constructors_csv[['constructorId', 'name']]
new_constructors_df.rename(columns = {'name':'Team Name'}, inplace = True)
new_constructors_df.head()

In [ ]:
#Merging mega2_merged_df, new_constructors_df on circuitId and ontaining final merged data
final_merged_df = pd.merge(new_constructors_df, mega2_merged_df, on=['constructorId'], how='outer')
final_merged_df = final_merged_df.sort_values('year', ascending=True)
final_merged_df = final_merged_df[['raceId', 'year', 'country', 'location', 'Circuit Name', 'Team Name', 'driverId',
                                   'driver name','laps', 'time', 'milliseconds', 'position', 'fastestLap', 'fastestLapTime',
                                   'fastestLapSpeed', 'constructorId', 'lat', 'lng']]
final_merged_df.drop_duplicates(inplace = True)
final_merged_df = final_merged_df.dropna()
final_merged_df = final_merged_df.reset_index(drop=True)
final_merged_df.head()

In [ ]:
# Calculating fastest/slowest speed of races from 2004
slowest_speed = final_merged_df["fastestLapSpeed"].min()
fastest_speed = final_merged_df["fastestLapSpeed"].max()

# Print
print(slowest_speed)
print(fastest_speed)

In [ ]:
# Calculating fastest/slowest lap times of races from 2004
fastest_laptime = final_merged_df["fastestLapTime"].min()
slowest_laptime = final_merged_df["fastestLapTime"].max()

# Print 
print(fastest_laptime)
print(slowest_laptime)

In [ ]:
# Setting 'year' as index and checking Top 3 fastest lap times
year_index_laptime_df = final_merged_df.set_index('year')
year_index_laptime_df = year_index_laptime_df.sort_values(('fastestLapTime'), ascending=True)
year_index_laptime_df.head(3)

In [ ]:
# Set the 'year' as our index and checking Top 3 fastest lap speeds
year_index_speed_df = final_merged_df.set_index('year')
year_index_speed_df = year_index_speed_df.sort_values(('fastestLapSpeed'), ascending=False)
year_index_speed_df.head(5)

In [ ]:
#Plotting Positions vs Fastest Lap Speed from 2004
fig, ax = plt.subplots(figsize=(8, 8))
ax.pie(final_merged_df['position'], labels=final_merged_df['fastestLapSpeed'], autopct='%1.1f%%', startangle=90)

ax.set_title('Positions vs Fastest Lap Speed')

plt.axis('equal')  

plt.tight_layout()
plt.show()

In [ ]:
# Build scatter plot for latitude vs. fastestLapSpeed
plt.scatter(final_merged_df['lat'], final_merged_df['fastestLapSpeed'], c='blue', s=50)

# Save the figure
# plt.savefig("OutputYK/LatitudeVSFastestLapSpeed.png")

# Show plot
plt.show()